In [3]:
from google.colab import files
import io
import pandas as pd
import numpy as np
import seaborn as sns
import  matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
uploaded = files.upload()

uploaded2 = files.upload()



In [35]:
titanic_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
#titanic_train = pd.read_csv('train.csv')

titanic_test = pd.read_csv(io.BytesIO(uploaded2['test.csv']))
#titanic_test = pd.read_csv('test.csv')

In [6]:
#data browsing
titanic_test.head()

titanic_train.info()

titanic_train = titanic_train.set_index('PassengerId')

titanic_train.head()

titanic_train.isnull().sum()

titanic_train.loc[titanic_train['Age'].isnull()]

titanic_train.groupby('Pclass').mean()['Survived']

titanic_train_num = titanic_train.select_dtypes(include=[np.number])
titanic_train_num.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Index(['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [7]:
#browsing the survival rate in each group
for j in ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']:
  print(titanic_train_num.groupby(j).mean()['Survived'])

titanic_train_obj = titanic_train.select_dtypes(exclude=[np.number])
titanic_train_obj

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64
Age
0.42     1.0
0.67     1.0
0.75     1.0
0.83     1.0
0.92     1.0
        ... 
70.00    0.0
70.50    0.0
71.00    0.0
74.00    0.0
80.00    1.0
Name: Survived, Length: 88, dtype: float64
SibSp
0    0.345395
1    0.535885
2    0.464286
3    0.250000
4    0.166667
5    0.000000
8    0.000000
Name: Survived, dtype: float64
Parch
0    0.343658
1    0.550847
2    0.500000
3    0.600000
4    0.000000
5    0.200000
6    0.000000
Name: Survived, dtype: float64
Fare
0.0000      0.066667
4.0125      0.000000
5.0000      0.000000
6.2375      0.000000
6.4375      0.000000
              ...   
227.5250    0.750000
247.5208    0.500000
262.3750    1.000000
263.0000    0.500000
512.3292    1.000000
Name: Survived, Length: 248, dtype: float64


,Name,Sex,Ticket,Cabin,Embarked
PassengerId,,,,,
1,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
3,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
5,"Allen, Mr. William Henry",male,373450,NaN,S
...,...,...,...,...,...
887,"Montvila, Rev. Juozas",male,211536,NaN,S
888,"Graham, Miss. Margaret Edith",female,112053,B42,S
889,"Johnston, Miss. Catherine Helen ""Carrie""",female,W./C. 6607,NaN,S


In [ ]:
for i in titanic_train_obj:
  print(titanic_train.groupby(i).mean()['Survived'])

Feature_list_obj = ['Sex','Embarked']

Feature_list_num = []
#---------above-----data scanning------#

In [9]:
#preprocessing data
X = titanic_train.drop(['Survived'], axis=1) #training data
y = titanic_train['Survived'] #prediction

X['AgeGroup']=pd.cut(X['Age'],[0,19,65,100], labels=['Minor','Adult','Elder']) #age catgorized

X = X.drop(['Age'], axis=1)



X_null_check = X.loc[X['AgeGroup'].isnull()] #Null check

X.groupby('AgeGroup').mean()


,Pclass,SibSp,Parch,Fare
AgeGroup,,,,
Minor,2.530488,1.115854,0.823171,31.794741
Adult,2.154982,0.335793,0.317343,35.675022
Elder,1.750000,0.125000,0.125000,27.710425


In [10]:
#Fill in NaN in ['AgeGroup']
for id in X_null_check.index:
  if X.loc[id,'SibSp'] >= 1  and X.loc[id, 'Parch'] >= 1:
    X.loc[id,'AgeGroup'] = 'Minor'
  elif X.loc[id,'SibSp'] >= 1 or X.loc[id, 'Parch'] >= 1:
    X.loc[id,'AgeGroup'] = 'Adult'
  else:
    X.loc[id,'AgeGroup'] = 'Elder'

In [11]:
#checking if any null
X.isnull().sum()

X = X.drop('Cabin', axis=1)

X = X.drop('Ticket', axis=1)

X = X.drop('Name', axis=1)

X.isnull().sum()

X.loc[X['Embarked'].isnull()]

mode = X['Embarked'].mode().values[0]
X['Embarked'] = X['Embarked'].fillna(value=mode) #FillNA with Mode

X.isnull().sum()

Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    0
AgeGroup    0
dtype: int64

In [ ]:
# Split Train and Validation set
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=1)

for i in X_train.columns:
  print(X_train[i].dtype != 'int64')

object_cols = [cols for cols in X.columns if (X[cols].dtype != 'int64') & (X[cols].dtype != 'float64')]

object_cols

In [13]:
def score_dataset(X_train, X_test, y_train, y_test):
  model = RandomForestRegressor()
  model.fit(X_train, y_train)
  preds = model.predict(X_test)
  return mean_absolute_error(y_test, preds)

label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

In [14]:
#Apply label encoder to each column with categorical data
#label_encoder
label_encoder = LabelEncoder()
for col in object_cols:
  label_X_train[col] = label_encoder.fit_transform(X_train[col])
  label_X_valid[col] = label_encoder.transform(X_valid[col])

print('MAE from label encoding:')
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

label_X_train #check value after numerized

MAE from label encoding:
0.25714771912426854


,Pclass,Sex,SibSp,Parch,Fare,Embarked,AgeGroup
PassengerId,,,,,,,
36,1,1,1,0,52.0000,2,0
47,3,1,1,0,15.5000,1,0
454,1,1,1,0,89.1042,0,0
292,1,0,1,0,91.0792,0,2
749,1,1,1,0,53.1000,2,2
...,...,...,...,...,...,...,...
716,3,1,0,0,7.6500,2,2
768,3,0,0,0,7.7500,1,0
73,2,1,0,0,73.5000,2,0


In [15]:
#testing and recording MAE in different n_estimators
MAE = {} 
for i in range(100,1001, 50):
  model = RandomForestRegressor(n_estimators=i, criterion='mse')
  model.fit(label_X_train, y_train)
  preds = model.predict(label_X_valid)
  print(f'n_estimators={i}:','MAE:', mean_absolute_error(y_valid, preds))
  MAE[f'n_estimators={i}'] = mean_absolute_error(y_valid, preds)

sorted_MAE_RFR = sorted(MAE.items(), key=lambda kv: kv[1])

sorted_MAE_RFR

n_estimators=100: MAE: 0.26020569691469875
n_estimators=150: MAE: 0.261679910457112
n_estimators=200: MAE: 0.2585447091133172
n_estimators=250: MAE: 0.2592734540974193
n_estimators=300: MAE: 0.25901007729874637
n_estimators=350: MAE: 0.2607335243970206
n_estimators=400: MAE: 0.2602442796770439
n_estimators=450: MAE: 0.2600676559611921
n_estimators=500: MAE: 0.26018571533534485
n_estimators=550: MAE: 0.2604790608701569
n_estimators=600: MAE: 0.25973495165408816
n_estimators=650: MAE: 0.26097424147654563
n_estimators=700: MAE: 0.2593455283039135
n_estimators=750: MAE: 0.2586424829432147
n_estimators=800: MAE: 0.2609751923980429
n_estimators=850: MAE: 0.25942298384341783
n_estimators=900: MAE: 0.2603649516473499
n_estimators=950: MAE: 0.26123350822911445
n_estimators=1000: MAE: 0.25954546285945224


[('n_estimators=200', 0.2585447091133172),
 ('n_estimators=750', 0.2586424829432147),
 ('n_estimators=300', 0.25901007729874637),
 ('n_estimators=250', 0.2592734540974193),
 ('n_estimators=700', 0.2593455283039135),
 ('n_estimators=850', 0.25942298384341783),
 ('n_estimators=1000', 0.25954546285945224),
 ('n_estimators=600', 0.25973495165408816),
 ('n_estimators=450', 0.2600676559611921),
 ('n_estimators=500', 0.26018571533534485),
 ('n_estimators=100', 0.26020569691469875),
 ('n_estimators=400', 0.2602442796770439),
 ('n_estimators=900', 0.2603649516473499),
 ('n_estimators=550', 0.2604790608701569),
 ('n_estimators=350', 0.2607335243970206),
 ('n_estimators=650', 0.26097424147654563),
 ('n_estimators=800', 0.2609751923980429),
 ('n_estimators=950', 0.26123350822911445),
 ('n_estimators=150', 0.261679910457112)]

In [16]:
#Selecting 200 as n_estimators parameter
model = RandomForestRegressor(n_estimators=200, criterion='mse')
model.fit(label_X_train, y_train)
preds = model.predict(label_X_valid)
print(f'n_estimators={i}:','MAE:', mean_absolute_error(y_valid, preds))

preds_round = preds.round()

#using confusion metrix to check the accuracy
cm = confusion_matrix(y_valid, preds_round)

cm

n_estimators=1000: MAE: 0.26245194731871013


array([[115,  13],
       [ 38,  57]])

In [17]:
#predcitions accuracy
correct_predicts = 0
total_predicts = 0
for i in range(len(y_valid)):
  if y_valid.iloc[i] == preds_round[i]:
    correct_predicts += 1
  total_predicts +=1
print('prediction accuracy = {}'.format(correct_predicts/total_predicts))

prediction accuracy = 0.7713004484304933


In [18]:
#------------Preprocessing "tes"t data---------------#

titanic_test = titanic_test.set_index('PassengerId')

titanic_test.shape

titanic_test['AgeGroup']=pd.cut(titanic_test['Age'],[0,19,65,100], labels=['Minor','Adult','Elder']) #age catgorized

titanic_test.isnull().sum()

label_X_train.columns

label_X_train_cols = []
for i in label_X_train.columns:
  label_X_train_cols.append(i)

titanic_test = titanic_test.drop('Age', axis=1)

T = titanic_test

T.loc[T['AgeGroup'].isnull()]

T_null_check = T.loc[T['AgeGroup'].isnull()] #Null check

T_null_check.index

#Fill in NaN in ['AgeGroup']
for id in T_null_check.index:
  if T.loc[id,'SibSp'] >= 1  and T.loc[id, 'Parch'] >= 1:
    T.loc[id,'AgeGroup'] = 'Minor'
  elif T.loc[id,'SibSp'] >= 1 or T.loc[id, 'Parch'] >= 1:
    T.loc[id,'AgeGroup'] = 'Adult'
  else:
    T.loc[id,'AgeGroup'] = 'Elder'

T.isnull().sum()

T.loc[T['Fare'].isnull()]

mean = T.groupby('Pclass').mean()['Fare'][3] #mean=average(Pclass 3)'s fare

T['Fare'] = T['Fare'].fillna(value=mean) #FillNA with mean

T.loc[1044]

T = T[label_X_train_cols] #keep training columns

label_T = T.copy()

In [19]:
# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
  label_X_train[col] = label_encoder.fit_transform(X_train[col])
  label_T[col] = label_encoder.transform(T[col])

label_T

preds_T = model.predict(label_T) #making predictions

preds_T

preds_T_round = preds_T.round()

preds_T_round

preds_T_int = preds_T_round.astype(int)

preds_T_int

output = pd.DataFrame({'PassengerId': label_T.index,
                       'Survived': preds_T_int})
output.to_csv('submission.csv', index=False)
files.download('submission.csv')
#-----------finish--------------#

In [20]:
#-------try less columns------#

label_X_train_less = label_X_train.drop(['SibSp','Parch','Fare'], axis=1)

label_X_train_less

label_X_valid_less = label_X_valid.drop(['SibSp','Parch','Fare'], axis=1)

In [ ]:
MAE_less = {} 
for i in range(100,1001, 50):
  model = RandomForestRegressor(n_estimators=i, criterion='mse')
  model.fit(label_X_train_less, y_train)
  preds = model.predict(label_X_valid_less)
  print(f'n_estimators={i}:','MAE:', mean_absolute_error(y_valid, preds))
  MAE_less[f'n_estimators={i}'] = mean_absolute_error(y_valid, preds)

In [ ]:
sorted_MAE_less_RFR = sorted(MAE_less.items(), key=lambda kv: kv[1])

sorted_MAE_less_RFR

In [29]:
model_700 = RandomForestRegressor(n_estimators=700, criterion='mse')
model_700.fit(label_X_train_less, y_train)
preds_700 = model_700.predict(label_X_valid_less)
print(mean_absolute_error(y_valid, preds_700))

0.2893424434426432


In [30]:
preds_700_round = preds_700.round()

In [31]:
#predcitions accuracy
correct_predicts_700 = 0
total_predicts_700 = 0
for i in range(len(y_valid)):
  if y_valid.iloc[i] == preds_700_round[i]:
    correct_predicts_700 += 1
  total_predicts_700 +=1
print('prediction accuracy = {}'.format(correct_predicts_700/total_predicts_700))

prediction accuracy = 0.7533632286995515


In [32]:
cm_700 = confusion_matrix(y_valid, preds_700_round)

In [33]:
cm_700

array([[114,  14],
       [ 41,  54]])

In [ ]:
#------------Preprocessing "test" data(LESS COLS)---------------#
titanic_test = titanic_test.set_index('PassengerId')

titanic_test.shape

titanic_test['AgeGroup']=pd.cut(titanic_test['Age'],[0,19,65,100], labels=['Minor','Adult','Elder']) #age catgorized

titanic_test.isnull().sum()

label_X_train.columns

label_X_train_cols_less = []
for i in label_X_train_less.columns:
  label_X_train_cols_less.append(i)

titanic_test = titanic_test.drop('Age', axis=1)

T = titanic_test

T.loc[T['AgeGroup'].isnull()]

T_null_check = T.loc[T['AgeGroup'].isnull()] #Null check

T_null_check.index


In [37]:
#Fill in NaN in ['AgeGroup']
for id in T_null_check.index:
  if T.loc[id,'SibSp'] >= 1  and T.loc[id, 'Parch'] >= 1:
    T.loc[id,'AgeGroup'] = 'Minor'
  elif T.loc[id,'SibSp'] >= 1 or T.loc[id, 'Parch'] >= 1:
    T.loc[id,'AgeGroup'] = 'Adult'
  else:
    T.loc[id,'AgeGroup'] = 'Elder'

T.isnull().sum()

T.loc[T['Fare'].isnull()]

mean = T.groupby('Pclass').mean()['Fare'][3] #mean=average(Pclass 3)'s fare

T['Fare'] = T['Fare'].fillna(value=mean) #FillNA with mean

T.loc[1044]

T_less = T[label_X_train_cols_less] #keep training columns

label_T_less = T_less.copy()


In [38]:
# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
  label_X_train_less[col] = label_encoder.fit_transform(X_train[col])
  label_T_less[col] = label_encoder.transform(T_less[col])

label_T_less

preds_T_less = model_700.predict(label_T_less) #making predictions

preds_T_less

preds_T_less_round = preds_T_less.round()

preds_T_less_round

preds_T_less_int = preds_T_less_round.astype(int)

preds_T_less_int

output = pd.DataFrame({'PassengerId': label_T_less.index,
                       'Survived': preds_T_less_int})
output.to_csv('submission.csv', index=False)